# Projet TAL M1 S1 : POS Tagging
structure du projet expliqué ici : https://u-paris.zoom.us/rec/share/rFB-gp7vkgMK31kfPxEsyNF2k6q7mHcq3P9MQiovimkr9ebIewZpuD62RBk6SmM.31Mmzx1VyMqQXRtu?startTime=1607605609000

# Implémentation du classifieur

## Import du corpus
Cours Zoom du 17/12 - 35 min

On charge les trois corpus *in-domain* sous la forme de listes de dictionnaires : chaque phrase a une clé "mots" qui est associée à une liste des mots, et une clé "POS" associée à une liste de POS. 

On utilise les trois corpus distincts de French-GSD :
- Train : apprentissage.
- Dev : validation. Pour tester et améliorer le modèle. 
- Test : évaluation. On ne l'utilisera pas pendant l'apprentissage ou le test.

In [1]:
# j'ai juste rajouté l'encodage car j'avais une erreur sans 

def load_corpus(file):
    with open(file, "r", encoding = "utf8") as f: 
        content = f.read() # chargement du corpus
    content = content.split("\n\n") # séparation en phrases
    corpus = []
    for phrase in content: # pour chaque phrase
        phrase_dico = {"mots" : [], "gold_labels" : []} # liste qui contiendra 1 dictionnaire par mot de la phrase
        for line in phrase.splitlines():
            if not line.startswith("#"): # on ignore les lignes qui commencent par #
                features = line.split("\t")
                phrase_dico["mots"].append(features[1])
                # phrase_dico["lemme"].append(features[2])
                phrase_dico["gold_labels"].append(features[3])
        corpus.append(phrase_dico)
    return corpus

gsd_train = load_corpus("corpus-in-domain/fr_gsd-ud-train.conllu")
gsd_test = load_corpus("corpus-in-domain/fr_gsd-ud-test.conllu")
gsd_dev = load_corpus("corpus-in-domain/fr_gsd-ud-dev.conllu")

In [2]:
print("---- Aperçus d'une phrase de chaque corpus-----", end="\n\n")
print(type(gsd_train[1]))
print(gsd_train[1], end="\n\n")
print(gsd_test[102], end="\n\n")
print(gsd_dev[564])

---- Aperçus d'une phrase de chaque corpus-----

<class 'dict'>
{'mots': ["L'", 'œuvre', 'est', 'située', 'dans', 'la', 'galerie', 'des', 'de', 'les', 'batailles', ',', 'dans', 'le', 'château', 'de', 'Versailles', '.'], 'gold_labels': ['DET', 'NOUN', 'AUX', 'VERB', 'ADP', 'DET', 'NOUN', '_', 'ADP', 'DET', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT']}

{'mots': ['La', 'gestion', 'et', "l'", 'exploitation', 'de', 'la', 'salle', 'de', 'concert', 'Wagram', ',', 'récemment', 'rénovée', ',', 'sera', 'assurée', 'par', 'Eurosites', ',', 'leader', 'en', 'France', 'de', 'la', 'location', 'de', 'salles', '.'], 'gold_labels': ['DET', 'NOUN', 'CCONJ', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'PROPN', 'PUNCT', 'ADV', 'VERB', 'PUNCT', 'AUX', 'VERB', 'ADP', 'PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'PUNCT']}

{'mots': ['Cette', 'espèce', 'est', 'endémique', 'du', 'de', 'le', 'département', 'de', 'Nariño', 'en', 'Colombie', '.'], 'gold_

## Extraction des caractéristiques

La fonction feature_extraction renvoie une liste de dictionnaires (un par mot) qui contiennent les caractéristiques suivantes pour chaque mot :
- mot
- mot précédent : pour le premier mot de la phrase son mot précédent sera "START", ce qui permettra de prendre en compte la caractéristique "être le premier mot".
- mot suivant : pour le dernier mot, ce sera "END".
- commence par une lettre majuscule
- est entièrement en majuscules
- contient des chiffres
- contient des caractères non alphanumériques
- longueur du mot (3 caractéristiques binaires) : a 1 seule caractère, a moins de 3 caractères, a plus de 3 caractères.
- a un suffixe nominal
- a un suffixe adjectival
- a un suffixe verbal (ça marche que sur les lemmes !)
- a un suffixe adverbial (là ça marche sur la forme fléchie)

Met-on dans les listes les suffixes qui appartiennent à 2 catégories ?
Peut-on utiliser les lemmes ?

On pourrait ajouter des suffixes aussi. Par exemple une feature pourrait être "finir en -ment", une autre suffi_nominaux (-al, -ion, etc.), etc. 
On est confrontées à un problème d'homonymie avec certains suffixes. Par exemple, -ains dans certains (pronom) et hautains (adj). A voir comment gérer ça. 

On ne conserve plus la structure des phrases, qui n'est plus nécessaire une fois qu'on a extrait les informations comme mot précédent et mot suivant.

In [3]:
def feature_extraction(corpus):

    corpus_features = []
    """
    Listes à utiliser pour les lemmes :
    list_adj = ["ain", "aine","aire","é", "ée","iel", "uel", "lle",  "al", "ales", "al", "ial","er", "ère", "ier", "esque", "eur", "euse", "ieux","ueux", "if","ive", "in","ine","ique","atoire", "u","ue", "issime", "able","ible", "uble", "ième","uple"]
    list_noun = ["ade", "age","aille", "aison", "ison","oison", "ation", "ition","ssion", "sion","xion", "isation","ment", "ement","erie", "ure","ature","at", "ance","ence", "escence","ité", "eté","té", "ie","erie", "esse","ise", "eur","isme", "iste", "seur", "isseur", "isateur", "euse","isseuse", "atrice","ier","ière", "aire","ien", ienne","iste",er","eron","eronne","trice","oir", "oire","ier","ière","erie","anderie", "aire","ain","aille", "ée", "ard","asse", "assier","âtre","aut","eau","ceau", "ereau","eteau", "elle", "et","elet","ette", 
"elette","in","otin", "ine" "illon","on", "ille", "erole","ole","iche"]
    """

    list_vb = ["iser","ifier", "oyer","ailler", "asser","eler", "eter","iller", "iner","nicher", "ocher","onner","otter","oter", "ouiller"]
    list_adj = ["ain", "aine","ains", "aines","aire", "aires","é", "ée","ées", "és","iel", "iels","uel", "uels", 
"lle", "lles","els", "el" "al", "ales", "al", "ial", "aux","iaux", "er","ers", "ère","ères", "ier", "iers",      "esque","esques", "eur","eurs", "euse","euses", "ieux","ueux", "if", "ifs","ive", "ives","in", "ins","ine",      "ines","iques", "ique","atoire", "u","ue", "us","ues", "issime","issimes","able","ible", "ibles","ables",  
    "uble","ubles", "ième","ièmes", "uple"]
    list_noun = ["ade", "ades", "age", "ages","aille", "ailles", "aison", "ison", "isons","oison", "ation", "itions", "ition", "ssion", "sion","xion", "isation","ment", "ement","erie", "eries","ure","ures","ature", "atures","at", "ance","ence", "escence","ité", "eté","té", "ie","erie", "esse", "ise", "eur","isme", "iste", "istes","eurs", "seur","seurs", "isseur","isseurs", "isateur","euse", "euses","isseuse", "isseuses",       "atrice", "atrices","ier", "iers","ière", "ières","aire","aires","ien", "iens","ienne", "iennes","iste",         "istes","er", "ers","eron", "erons","eronne","trice","oir", "oire","oires", "oirs","ier", "iers","ière",         "ières","erie","eries","anderie","aire", "aires","ain", "aines", "ée","ées","aille", "ard","asse", "asses", "assier","âtre","aut","eau", "eaux","ceau", "ereau","eteau", "elle","elles", "et","elet","ets","ette","elette","ettes", "elettes","in", "ins","otin", "ine","ines", "illon","on","ons","ille", "erole","eroles", "ole","oles", "iche"]

    for phrase in corpus: # ajout des features additionnelles
        for prev, word, suiv in zip(["START"] + phrase["mots"][:-1], phrase["mots"], phrase["mots"][1:] + ["END"]):
            # création de triplets (mot précédent, mot, mot suivant)
            # avec "START" en prev pour le 1er mot
            # et "END" en suiv pour le dernier

            # lemma = phrase["lemme"][phrase["mots"].index(word)] 

            # dictionnaire de features du mot
            features_mot = { 
                # on récupère le gold_label correspondant
                f"mot - {word}" : 1,
                f"prec - {prev}" : 1,
                f"mot_suiv - {suiv}" : 1,
                }
            
            if word.istitle(): features_mot["maj"] = 1 
            if word.isupper(): features_mot["all_caps"] = 1
            if any(char.isdigit() for char in word): features_mot["num"] = 1# mieux que isnumeric(), car renvoie false si espace (40 000) ou virgule (50,6) par ex
            if not word.isalnum(): features_mot["nonAlphanum"] = 1
            if len(word) <= 3: features_mot["court"] = 1 
            if len(word) > 3: features_mot["long"] = 1
            if len(word) == 1: features_mot["un_car"] = 1
            if word.endswith("ment"): features_mot["suff_adv"] = 1
            if any(word.endswith(elem) and len(word) != len(elem) for elem in list_noun): features_mot["suff_noun"] =1 
            if any(word.endswith(elem) and len(word) != len(elem) for elem in list_adj): features_mot["suff_adj"] = 1
            if any(word.endswith(elem) for elem in list_vb): features_mot["suff_vb"] = 1
            # on vérifie la longueur du mot pour être sûr que ce soit un suffixe car on peut avoir le mot                      age avec le suffixe age par exemple ou bien aux
            # suff_noun : any(lemma.endswith(elem) and len(word) != len(elem) for elem in list_noun),
            # suff_adj : any(lemma.endswith(elem) for elem in list_adj),
            # suff_vb : any(lemma.endswith(elem) for elem in list_vb)
            
            # ajout au corpus
            corpus_features.append(features_mot)

    return corpus_features # renvoie les features transformés en vecteurs one-hot


gsd_train_features = feature_extraction(gsd_train)

In [4]:
print(len(gsd_train_features))
print(*gsd_train_features[10:15], sep="\n")

364349
{"mot - qu'": 1, 'prec - sport': 1, 'mot_suiv - on': 1, 'nonAlphanum': 1, 'court': 1}
{'mot - on': 1, "prec - qu'": 1, 'mot_suiv - les': 1, 'court': 1}
{'mot - les': 1, 'prec - on': 1, 'mot_suiv - considére': 1, 'court': 1}
{'mot - considére': 1, 'prec - les': 1, 'mot_suiv - presque': 1, 'long': 1}
{'mot - presque': 1, 'prec - considére': 1, 'mot_suiv - comme': 1, 'long': 1, 'suff_adj': 1}


In [9]:
def add_gold(features, corpus):
    '''Ajoute les gold labels pour créer un corpus d'entraînement / de test'''
    i = 0
    gold_corpus = []
    for phrase in corpus:
        for word, pos_gold in zip(phrase["mots"],phrase["gold_labels"]):
            gold_corpus.append((features[i], pos_gold))
            i += 1

    return gold_corpus

gsd_train_features_gold = add_gold(gsd_train_features, gsd_train)

In [10]:
print(*gsd_train_features_gold[100:110], sep="\n")

({'mot - avec': 1, 'prec - reviendrais': 1, 'mot_suiv - plaisir': 1, 'long': 1}, 'ADP')
({'mot - plaisir': 1, 'prec - avec': 1, 'mot_suiv - !': 1, 'long': 1}, 'NOUN')
({'mot - !': 1, 'prec - plaisir': 1, 'mot_suiv - END': 1, 'nonAlphanum': 1, 'court': 1, 'un_car': 1}, 'PUNCT')
({'mot - Les': 1, 'prec - START': 1, 'mot_suiv - forfaits': 1, 'maj': 1, 'court': 1}, 'DET')
({'mot - forfaits': 1, 'prec - Les': 1, 'mot_suiv - comprennent': 1, 'long': 1}, 'NOUN')
({'mot - comprennent': 1, 'prec - forfaits': 1, 'mot_suiv - le': 1, 'long': 1}, 'VERB')
({'mot - le': 1, 'prec - comprennent': 1, 'mot_suiv - transport': 1, 'court': 1}, 'DET')
({'mot - transport': 1, 'prec - le': 1, 'mot_suiv - en': 1, 'long': 1}, 'NOUN')
({'mot - en': 1, 'prec - transport': 1, 'mot_suiv - car': 1, 'court': 1}, 'ADP')
({'mot - car': 1, 'prec - en': 1, 'mot_suiv - Grand': 1, 'court': 1}, 'NOUN')


## Implémentation de l'algorithme de classification
On a choisi d'implémenter la classification avec un perceptron moyenné.

Sources : cours 3/12 + perceptron.pdf + zoom du 17/12

La fonction predict sera utilisée à la fois dans l'apprentissage et dans la "prédiction". Elle correspond à la recherche de l'étiquette avec le plus grand score (argmax...)

In [49]:
def predict(word_features, weights):
    """Renvoie l'étiquette avec le plus gros score (argmax)"""
    scores = {}
    for tag, w in weights.items():
        scores[tag] =  sum(word_features.get(feat)*w.get(feat, 0) for feat in word_features)
    #print(scores)

    return max(scores, key=scores.get)

In [71]:
from random import shuffle

def perceptron_train(training_set, voc, MAX_EPOCH=3): # ça serait une bonne idée de faire une classe pour éviter d'avoir à redonner le vocabulaire en argument à chaque fois par exemple
    
    tags = ["ADJ", "ADP", "ADV", "AUX", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "PUNCT", "SCONJ", "SYM", "VERB", "X"]

    # initialisation de a (poids totaux)
    a = {}
    for tag in tags:
        a[tag] = {}

    # initialisation des vecteurs de poids
    w = {}
    for tag in tags:
        w[tag] = {}
    
    for i in range(0, MAX_EPOCH):
        shuffled_set = training_set.copy() # copie du training set
        shuffle(shuffled_set) # mélange du training set

        for word in shuffled_set:
            vec = word[0]
            gold = word[1]
          #  print("Scores avant modif")
            prediction = predict(vec, w) # trouve étiquette plus probable
           # print(f"\"{vec}\"\t{prediction}\t{gold}\t")
            if not gold == "_" and not prediction == gold: # si le gold_label n'est pas égal à celui prédit
                # on ignore les mots dont le gold_label est "_" ("au" et "du") car ils sont ensuite analysés comme (à le, de le)
                for feat in vec: # pour chaque feature du mot
                    w[gold][feat] = w[gold].get(feat,0) + vec.get(feat) #  on ajoute x(i) à chaque poids de l'étiquette correcte
                    w[prediction][feat] = w[prediction].get(feat,0) - vec[feat] #  on retire x(i) à chaque poids de l'étiquette mal prédite
        # on ajoute w à a
               # print("Scores après modif")
                predict(vec, w)
         #   print("\n")

        for tag in w:
            for index in w[tag].keys(): 
                a[tag][index] = a[tag].get(index, 0) + w[tag][index]

    return a


In [69]:
poids_gsd_train = perceptron_train(gsd_train_features_gold, gsd_train_features)

## Evaluation des performances sur le corpus de validation (Dev)
Il faudra bien tester les hyper paramètres (epoch...) : idée : on pourrait récupérer les poids toutes les x itérations, plutôt que de tout recommencer au début pour tester différents max_epoch.

In [73]:
gsd_dev_features = feature_extraction(gsd_dev)
gsd_dev_voc = add_gold(gsd_dev_features, gsd_dev)


def test(corpus, poids):
    """Prédit les étiquettes et renvoie un taux d'erreur"""
    nb_erreurs = 0
    output = open("test.txt", "w")
    output.write("MOT\tPREDICT\tGOLD\tRESULTAT\n")
    for word in corpus:
        vec = word[0]
        gold = word[1]
        prediction = predict(vec, poids)
       # print(f"{vec}\t{prediction}\t{gold}\t")
        if not gold == "_" and not prediction == gold:
            nb_erreurs +=1
            output.write("ERREUR")
        else:
            output.write("OK")
        output.write("\n")
    
    output.write(f"Nombre d\'erreurs : {nb_erreurs}")
    output.write(f"\nTaux d\'erreur : {nb_erreurs/len(corpus)}")
    output.close()

    return nb_erreurs/len(corpus)

print(test(gsd_dev_voc, poids_gsd_train))

0.04394289598912304


## Evaluation sur le corpus Test

# Evaluation hors-domaine

## Analyse de l'impact du changement de domaine
* identifier causes de la baisse de performance : analyse de sortie, matrice de confusion, erreurs les + fréquentes
* réfléchir à des caractéristiques plus adaptées
* sélection d'un nouvel ensemble d'apprentissage avec des exemples représentatif (généré par un modèle de langue type TP2)

### Corpus UGC (User-generated content)

### Corpus littéraire

## Développement de systèmes robutes au changement de domaine